In [26]:
import numpy as np 
import pandas as pd 
import seaborn as sns
pd.options.display.float_format = '{:.2f}'.format
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args
import pickle
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/dataset-final/X_copy.pkl
/kaggle/input/dataset-final/y_copy.pkl
/kaggle/input/new11111/Data_withbore.csv


In [5]:
with open('/kaggle/input/dataset-final/X_copy.pkl', 'rb') as f:
    X_copy = pickle.load(f)

with open('/kaggle/input/dataset-final/y_copy.pkl', 'rb') as f:
    y_copy = pickle.load(f)

In [6]:
def make_predictions(X_copy ,y_copy,ex , scaler):
    accuracy_scores = []
    classification_reports = []
    predictions = []
    dim_predictions = []
    prediction_probabilities = []

    ex = scaler.transform(ex)
    
    for i in range(16):
        X_train, X_test, y_train, y_test = train_test_split(X_copy.iloc[:69], y_copy.iloc[:69], test_size=0.2, random_state=i)

        X_train = X_train.reset_index(drop=True)
        X_test = X_test.reset_index(drop=True)
        y_train = y_train.reset_index(drop=True)
        y_test = y_test.reset_index(drop=True)
        xgb_classifier = XGBClassifier(random_state=4, max_depth=3, learning_rate=0.214, objective='binary:logistic')

        xgb_classifier.fit(X_train, y_train)

        y_pred = xgb_classifier.predict(X_test)
        y_new = xgb_classifier.predict(ex)
        y_prob = xgb_classifier.predict_proba(ex)[:, 1]
        
        predictions.append(y_new[0])
        prediction_probabilities.append(y_prob[0])
        
        accuracy = accuracy_score(y_test, y_pred)
        report = classification_report(y_test, y_pred)
        accuracy_scores.append(accuracy)
        classification_reports.append(report)

    mean_accuracy = np.mean(accuracy_scores)
    std_accuracy = np.std(accuracy_scores)

    mean_prediction = np.mean(predictions)
    mean_probability = np.mean(prediction_probabilities)
    ans = 1 if mean_prediction > 0.5 else 0
    i=1
    if(ans == 1):
        dim_predictions.append(ex)
        print(f"{i} Optimal Bushes found ")
        i+=1
    else:
        print("Current Bush not fine")
    return dim_predictions, mean_probability

In [34]:
def get_user_inputs():
    user_input = {}
    
    user_input['Tons'] = float(input("Enter Tons: "))
    user_input['Bush Length'] = float(input("Enter Bush Length: "))
    user_input['WP'] = float(input("Enter WP: "))
    user_input['Bore'] = float(input("Enter Bore: "))
    user_input['Rod'] = float(input("Enter Rod: "))
    user_input['Stroke'] = float(input("Enter Stroke: "))
    user_input['Type'] = int(input("Enter Type: "))
    
    Bush_OD_min = float(input("Enter minimum Bush OD: "))
    Bush_OD_max = float(input("Enter maximum Bush OD: "))
    
    clear_min = float(input("Enter minimum Clearance: "))
    clear_max = float(input("Enter maximum Clearance: "))
    
    Notches_min = int(input("Enter minimum Notches: "))
    Notches_max = int(input("Enter maximum Notches: "))
    
    Notch_angle_degree_min = float(input("Enter minimum Notch Angle degree: "))
    Notch_angle_degree_max = float(input("Enter maximum Notch Angle degree: "))
    
    Notch_length_min = float(input("Enter minimum Notch length (mm): "))
    Notch_length_max = user_input['Bush Length']  # This remains dependent on the user input for Bush Length
    
    return user_input, Bush_OD_min, Bush_OD_max, clear_min, clear_max, Notches_min, Notches_max, Notch_angle_degree_min, Notch_angle_degree_max, Notch_length_min, Notch_length_max

# Example usage
user_input, Bush_OD_min, Bush_OD_max, clear_min, clear_max, Notches_min, Notches_max, Notch_angle_degree_min, Notch_angle_degree_max, Notch_length_min, Notch_length_max = get_user_inputs()

search_space = [
    Real(Bush_OD_min, Bush_OD_max, name='Bush OD'),
    Real(clear_min, clear_max, name='Clearance'),
    Integer(Notches_min, Notches_max, name='Notches'),
    Real(Notch_angle_degree_min, Notch_angle_degree_max, name='Notch Angle degree'),
    Real(Notch_length_min, Notch_length_max, name='Notch length(mm)')  # Assuming 'Bush Length' is 35
]

@use_named_args(search_space)
def objective(**params):
    sample = {
        'WP': 240,  
        'Bore ': 80,  
        'Stroke': 698.5, 
        'Clearance': params['Clearance'],
        'Bush OD': params['Bush OD'],
        'Notch Angle degree': params['Notch Angle degree'],
        'Notches': params['Notches'],
        'Notch length(mm)': params['Notch length(mm)'],
        'Rod': 50, 
    }
    sample_df = pd.DataFrame([sample])

    sample_df['diff'] = (sample_df['Notches'] - sample_df['Notch length(mm)'])**2
    sample_df['Clearance_area'] = np.pi * (2 * sample_df['Bush OD'] * sample_df['Clearance'] + (sample_df['Clearance'])**2)
    
    sample_df_test = sample_df.drop(['Notches', 'Notch length(mm)', 'Rod', 'Bush OD'], axis=1)

    dim_predictions, mean_probability = make_predictions(X_copy , y_copy , sample_df_test , scaler)
    
    return -mean_probability # Minimize the negative of the mean probability for optimization

samples = 10
scaler = StandardScaler()
X_copy = pd.DataFrame(scaler.fit_transform(X_copy),columns = X_copy.columns)

res = gp_minimize(objective, search_space, n_calls=samples, random_state=0)

# Best result
print("Best parameters found:")
print(res.x)
print("Best objective value:")
print(-res.fun)


Enter Tons:  1.9
Enter Bush Length:  25
Enter WP:  240
Enter Bore:  70
Enter Rod:  40
Enter Stroke:  490
Enter Type:  0
Enter minimum Bush OD:  21.5
Enter maximum Bush OD:  22.3
Enter minimum Clearance:  0.15
Enter maximum Clearance:  0.19
Enter minimum Notches:  2
Enter maximum Notches:  4
Enter minimum Notch Angle degree:  0.5
Enter maximum Notch Angle degree:  1.5
Enter minimum Notch length (mm):  21


1 Optimal Bushes found 
1 Optimal Bushes found 
1 Optimal Bushes found 
1 Optimal Bushes found 
1 Optimal Bushes found 
1 Optimal Bushes found 
1 Optimal Bushes found 
1 Optimal Bushes found 
1 Optimal Bushes found 
1 Optimal Bushes found 
Best parameters found:
[21.974275694580015, 0.1837706299432407, 4, 1.3472517387841256, 23.49425478714389]
Best objective value:
0.5701287


In [35]:
user_input['Bush OD'] = res.x[0]
user_input['Clearance'] = res.x[1]
user_input['Notches'] = res.x[2]
user_input['Notch Angle degree'] = res.x[3]
user_input['Notch length(mm)'] = res.x[4]

In [36]:
output = user_input

In [39]:
output

{'Tons': 1.9,
 'Bush Length': 25.0,
 'WP': 240.0,
 'Bore': 70.0,
 'Rod': 40.0,
 'Stroke': 490.0,
 'Type': 0,
 'Bush OD': 21.974275694580015,
 'Clearance': 0.1837706299432407,
 'Notches': 4,
 'Notch Angle degree': 1.3472517387841256,
 'Notch length(mm)': 23.49425478714389}